In [3]:
from datetime import datetime
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
from time import sleep
import os


#import sys 
#sys.path.append("../")   aquí no es necesario porque tengo el src a la misma altura que el jupyter

from src.funciones import info_vuelos, crearDF_formateado, info_hoteles, crearDF_formateado_hoteles



Geopy no lo suo al final

In [ ]:
# geolocator = Nominatim(user_agent="my_application") #Primero hice un ejemplo con uno
# location = geolocator.geocode("santa cruz de tenerife")
# print(location.address)
# print((location.latitude, location.longitude))
# print(location.raw)

# lat= location.latitude
# lng= location.longitude
# print(lat)
# print(lng)

**API de Hoteles en TENE**

In [20]:
url = "https://tripadvisor-scraper.p.rapidapi.com/hotels/list"

querystring = {"query":"santa cruz de tenerife","page":"1"}

headers = {
	"x-rapidapi-key": "978309fe9dmshe0626a77c211be6p1786a7jsne7e8845b8cfa",
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

answer=response.json()

In [5]:
with open('hoteltenerife.json', 'w') as archivo:
    json.dump(answer, archivo, indent=4)

In [75]:
# nombre_hotel=answer["results"][0]["name"]
# reviews_hotel=answer["results"][0]["reviews"]
# rating_hotel=answer["results"][0]["rating"]
# precio_rango_min=answer["results"][0]["price_range_usd"]["min"]
# precio_rango_max=answer["results"][0]["price_range_usd"]["max"]
# precio_medio= (precio_rango_max + precio_rango_min)/2
# ciudad=answer["results"][0]["detailed_address"]["city"]
# ranking_sobre_20=answer["results"][0]["ranking"]["current_rank"]   #el ranking lo he quitado
# servicios=answer["results"][0]["amenities"]
# link=answer["results"][0]["link"]

In [76]:

# diccht= {"nombre_hotel":nombre_hotel,
#        "reviews_hotel":reviews_hotel,
#        "rating_hotel": rating_hotel,
#        "precio_rango_min":precio_rango_min,
#        "precio_rango_max":precio_rango_max,
#        "precio_medio_por noche":precio_medio,
#        "ciudad":ciudad,
#        "ranking_(/20)":ranking_sobre_20,   #el ranking lo he quitado
#        "servicios":servicios,
#        "link":link
#         }
# diccht

{'nombre_hotel': 'Iberostar Heritage Grand Mencey',
 'reviews_hotel': 2829,
 'rating_hotel': 4.5,
 'precio_rango_min': 133,
 'precio_rango_max': 212,
 'precio_medio_por noche': 172.5,
 'ciudad': 'Santa Cruz de Tenerife',
 'ranking_(/20)': 1,
 'servicios': ['Free Wifi',
  'Pool',
  'Outdoor pool',
  'Restaurant',
  'Room service'],
 'link': 'https://www.tripadvisor.com/Hotel_Review-g187482-d1555517-Reviews-Iberostar_Heritage_Grand_Mencey-Santa_Cruz_de_Tenerife_Tenerife_Canary_Islands.html'}

Sacar hoteles TENE:

In [6]:
# lista_diccht=[]

# for i in range(0, len(answer["results"])):

#         nombre_hotel=answer["results"][i]["name"]
#         reviews_hotel=answer["results"][i]["reviews"]
#         rating_hotel=answer["results"][i]["rating"]
#         precio_rango_min=answer["results"][i]["price_range_usd"]["min"]
#         precio_rango_max=answer["results"][i]["price_range_usd"]["max"]
#         precio_medio= (precio_rango_max + precio_rango_min)/2
#         ciudad=answer["results"][i]["detailed_address"]["city"]
#         servicios=answer["results"][i]["amenities"]
#         link=answer["results"][i]["link"]

        
#         diccht= {"nombre_hotel":nombre_hotel,
#                 "reviews_hotel":reviews_hotel,
#                 "rating_hotel": rating_hotel,
#                 "precio_rango_min":precio_rango_min,
#                 "precio_rango_max":precio_rango_max,
#                 "precio_medio_por noche":precio_medio,
#                 "ciudad":ciudad,
#                 "servicios":servicios,
#                 "link":link
#                 }



#         lista_diccht.append(diccht)

In [7]:
# df_hotel_tenerife= pd.DataFrame(lista_diccht).drop(columns= ["precio_rango_min", "precio_rango_max"])
# df_hotel_tenerife.head(3)

,nombre_hotel,reviews_hotel,rating_hotel,precio_medio_por noche,ciudad,servicios,link
0,Iberostar Heritage Grand Mencey,2829,4.5,172.5,Santa Cruz de Tenerife,"[Free Wifi, Pool, Outdoor pool, Restaurant, Ro...",https://www.tripadvisor.com/Hotel_Review-g1874...
1,AC Hotel Tenerife,70,3.5,139.0,Santa Cruz de Tenerife,"[Free Wifi, Free parking, Breakfast included, ...",https://www.tripadvisor.com/Hotel_Review-g1874...
2,NH Tenerife,866,4.0,115.0,Santa Cruz de Tenerife,"[Restaurant, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g1874...


In [8]:
# df_hotel_tenerife.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            33 non-null     object 
 1   reviews_hotel           33 non-null     int64  
 2   rating_hotel            33 non-null     float64
 3   precio_medio_por noche  33 non-null     float64
 4   ciudad                  33 non-null     object 
 5   servicios               33 non-null     object 
 6   link                    33 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.9+ KB


**API de Hoteles en TENE Done**

In [9]:
# df_hotel_tenerife.head(2)

,nombre_hotel,reviews_hotel,rating_hotel,precio_medio_por noche,ciudad,servicios,link
0,Iberostar Heritage Grand Mencey,2829,4.5,172.5,Santa Cruz de Tenerife,"[Free Wifi, Pool, Outdoor pool, Restaurant, Ro...",https://www.tripadvisor.com/Hotel_Review-g1874...
1,AC Hotel Tenerife,70,3.5,139.0,Santa Cruz de Tenerife,"[Free Wifi, Free parking, Breakfast included, ...",https://www.tripadvisor.com/Hotel_Review-g1874...


**API de Hoteles en PMI**

In [44]:


url_hotel = "https://tripadvisor-scraper.p.rapidapi.com/hotels/list"

querystring = {f"query":"palma de mallorca","page":"8"}

headers = {
	"x-rapidapi-key": "978309fe9dmshe0626a77c211be6p1786a7jsne7e8845b8cfa",
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url_hotel, headers=headers, params=querystring)
respond=response.json()


Esto para guardar en archivo csv

In [47]:
# info_hoteles(respond)     #Lo menciono para que NO MOLESTE

In [46]:
# df=crearDF_formateado_hoteles(info_hoteles(respond))
# df.to_csv('datos/hotel_pmi_8.csv', index=False)


In [11]:
with open('hotelmallorca.json', 'w') as archivo:
    json.dump(respond, archivo, indent=4)

In [45]:
# nombre_hotel=respond["results"][0]["name"]
# reviews_hotel=respond["results"][0]["reviews"]
# rating_hotel=respond["results"][0]["rating"]
# precio_rango_min=respond["results"][0]["price_range_usd"]["min"]
# precio_rango_max=respond["results"][0]["price_range_usd"]["max"]
# precio_medio= (precio_rango_max + precio_rango_min)/2
# ciudad=respond["results"][0]["detailed_address"]["city"]
# ranking_sobre_95=respond["results"][0]["ranking"]["current_rank"]     #el ranking lo he quitado
# servicios=respond["results"][0]["amenities"]
# link=respond["results"][0]["link"]

In [46]:

# dicch= {"nombre_hotel":nombre_hotel,
#        "reviews_hotel":reviews_hotel,
#        "rating_hotel": rating_hotel,
#        "precio_rango_min":precio_rango_min,
#        "precio_rango_max":precio_rango_max,
#        "precio_medio_por noche":precio_medio,
#        "ciudad":ciudad,
#        "ranking_(/95)":ranking_sobre_95,   #el ranking lo he quitado
#        "servicios":servicios,
#        "link":link
#         }
# dicch

{'nombre_hotel': 'Posada Terra Santa',
 'reviews_hotel': 1695,
 'rating_hotel': 5,
 'precio_rango_min': 248,
 'precio_rango_max': 391,
 'precio_medio_por noche': 319.5,
 'ciudad': 'Palma de Mallorca',
 'ranking_(/95)': 2,
 'servicios': ['Free Wifi',
  'Breakfast included',
  'Pool',
  'Outdoor pool',
  'Restaurant'],
 'link': 'https://www.tripadvisor.com/Hotel_Review-g187463-d6479649-Reviews-Posada_Terra_Santa-Palma_de_Mallorca_Majorca_Balearic_Islands.html'}

Sacar los hoteles de PMI:

In [12]:
# lista_dicch=[]

# for i in range(0,len(respond["results"])):

#         nombre_hotel=respond["results"][i]["name"]
#         reviews_hotel=respond["results"][i]["reviews"]
#         rating_hotel=respond["results"][i]["rating"]
#         precio_rango_min=respond["results"][i]["price_range_usd"]["min"]
#         precio_rango_max=respond["results"][i]["price_range_usd"]["max"]
#         precio_medio= (precio_rango_max + precio_rango_min)/2
#         ciudad=respond["results"][i]["detailed_address"]["city"]
#         servicios=respond["results"][i]["amenities"]
#         link=respond["results"][i]["link"]

        
#         dicch= {"nombre_hotel":nombre_hotel,
#                 "reviews_hotel":reviews_hotel,
#                 "rating_hotel": rating_hotel,
#                 "precio_rango_min":precio_rango_min,
#                 "precio_rango_max":precio_rango_max,
#                 "precio_medio_por noche":precio_medio,
#                 "ciudad":ciudad,
#                 "servicios":servicios,
#                 "link":link
#                 }



#         lista_dicch.append(dicch)

In [13]:
# df_hotel_mallorca= pd.DataFrame(lista_dicch).drop(columns= ["precio_rango_min", "precio_rango_max"])
# df_hotel_mallorca.head(4)

,nombre_hotel,reviews_hotel,rating_hotel,precio_medio_por noche,ciudad,servicios,link
0,Puro Hotel Palma,1131,4.5,253.0,Palma de Mallorca,"[Free Wifi, Breakfast included, Restaurant, Ro...",https://www.tripadvisor.com/Hotel_Review-g1874...
1,Posada Terra Santa,1695,5.0,319.5,Palma de Mallorca,"[Free Wifi, Breakfast included, Pool, Outdoor ...",https://www.tripadvisor.com/Hotel_Review-g1874...
2,"Castillo Hotel Son Vida, a Luxury Collection H...",1203,4.5,417.5,Palma de Mallorca,"[Free Wifi, Free parking, Breakfast included, ...",https://www.tripadvisor.com/Hotel_Review-g1874...
3,Calatrava Mediterranean Sea House,830,5.0,433.5,Palma de Mallorca,"[Free Wifi, Restaurant, Airport transportation...",https://www.tripadvisor.com/Hotel_Review-g1874...


In [14]:
# df_hotel_mallorca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            30 non-null     object 
 1   reviews_hotel           30 non-null     int64  
 2   rating_hotel            30 non-null     float64
 3   precio_medio_por noche  30 non-null     float64
 4   ciudad                  30 non-null     object 
 5   servicios               30 non-null     object 
 6   link                    30 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.8+ KB


**API de Hoteles en PMI Done**

In [15]:
# df_hotel_mallorca.head(2)

,nombre_hotel,reviews_hotel,rating_hotel,precio_medio_por noche,ciudad,servicios,link
0,Puro Hotel Palma,1131,4.5,253.0,Palma de Mallorca,"[Free Wifi, Breakfast included, Restaurant, Ro...",https://www.tripadvisor.com/Hotel_Review-g1874...
1,Posada Terra Santa,1695,5.0,319.5,Palma de Mallorca,"[Free Wifi, Breakfast included, Pool, Outdoor ...",https://www.tripadvisor.com/Hotel_Review-g1874...


**Funciones hoteles(sin pasadas a .py):**

In [13]:
def info_hoteles(request_jeison):
    lista_dicch=[]
    respond=request_jeison
    for i in range(0,len(respond["results"])):

        nombre_hotel=respond["results"][i]["name"]
        reviews_hotel=respond["results"][i]["reviews"]
        rating_hotel=respond["results"][i]["rating"]
        precio_rango_min=respond["results"][i]["price_range_usd"]["min"]
        precio_rango_max=respond["results"][i]["price_range_usd"]["max"]
        precio_medio= (precio_rango_max + precio_rango_min)/2
        ciudad=respond["results"][i]["detailed_address"]["city"]
        servicios=respond["results"][i]["amenities"]
        link=respond["results"][i]["link"]

        
        dicch= {"nombre_hotel":nombre_hotel,
                "reviews_hotel":reviews_hotel,
                "rating_hotel": rating_hotel,
                "precio_rango_min":precio_rango_min,
                "precio_rango_max":precio_rango_max,
                "precio_medio_por noche":precio_medio,
                "ciudad":ciudad,
                "servicios":servicios,
                "link":link
                }
        lista_dicch.append(dicch)
    return lista_dicch
        

In [21]:
info_hoteles(respond)     #Lo menciono para que NO MOLESTE

In [17]:
def crearDF_formateado_hoteles(funcion_info_vuelos):

    df_hotel= pd.DataFrame(funcion_info_vuelos).drop(columns= ["precio_rango_min", "precio_rango_max"])
    df_hotel
    return df_hotel

In [20]:
crearDF_formateado_hoteles(info_hoteles(respond)).head()   

,nombre_hotel,reviews_hotel,rating_hotel,precio_medio_por noche,ciudad,servicios,link
0,Puro Hotel Palma,1131,4.5,253.0,Palma de Mallorca,"[Free Wifi, Breakfast included, Restaurant, Ro...",https://www.tripadvisor.com/Hotel_Review-g1874...
1,Posada Terra Santa,1695,5.0,319.5,Palma de Mallorca,"[Free Wifi, Breakfast included, Pool, Outdoor ...",https://www.tripadvisor.com/Hotel_Review-g1874...
2,"Castillo Hotel Son Vida, a Luxury Collection H...",1203,4.5,417.5,Palma de Mallorca,"[Free Wifi, Free parking, Breakfast included, ...",https://www.tripadvisor.com/Hotel_Review-g1874...
3,Calatrava Mediterranean Sea House,830,5.0,433.5,Palma de Mallorca,"[Free Wifi, Restaurant, Airport transportation...",https://www.tripadvisor.com/Hotel_Review-g1874...
4,Nivia Born Boutique Hotel,119,4.5,338.0,Palma de Mallorca,"[Free Wifi, Restaurant, Room service, Pets All...",https://www.tripadvisor.com/Hotel_Review-g1874...


**API de los vuelos(TENE (TNF Y TNS))**  Meter los códigos de aeropuerto que necesitemos

In [3]:
url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {"originSkyId":"MAD","destinationSkyId":"TENE","originEntityId":"95565077","destinationEntityId":"27547165","date":"2024-11-03","returnDate":"2024-11-06","cabinClass":"economy","adults":"1","sortBy":"best","currency":"EUR"}

headers = {
	"x-rapidapi-key": "978309fe9dmshe0626a77c211be6p1786a7jsne7e8845b8cfa",
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

respuesta=response.json()



In [83]:
# with open('tenerife.json', 'w') as archivo:
#     json.dump(respuesta, archivo, indent=4)

In [84]:
# resp= respuesta["data"]
# precio= resp["itineraries"][0]["price"]["raw"]

# id_origen= resp["itineraries"][0]["legs"][0]["origin"]["id"]
# origen= resp["itineraries"][0]["legs"][0]["origin"]["city"]

# id_destino= resp["itineraries"][0]["legs"][0]["destination"]["id"]
# destino= resp["itineraries"][0]["legs"][0]["destination"]["city"]

# duracion_minuts= resp["itineraries"][0]["legs"][0]["durationInMinutes"]
# escalas= resp["itineraries"][0]["legs"][0]["stopCount"]

# salida= resp["itineraries"][0]["legs"][0]["departure"]
# llegada= resp["itineraries"][0]["legs"][0]["arrival"]

# aerolinea= resp["itineraries"][0]["legs"][0]["carriers"]["marketing"][0]["name"]

In [85]:

# dicct= {"id_origen":id_origen,
#        "ciudad_origen":origen,
#        "id_destino": id_destino,
#        "ciudad_destino":destino,
#        "salida":salida,
#        "llegada":llegada,
#        "duracion(min)":duracion_minuts,
#        "escalas":escalas,
#        "aerolinea":aerolinea,
#        "precio":precio
#         }
# dicct

{'id_origen': 'MAD',
 'ciudad_origen': 'Madrid',
 'id_destino': 'TFN',
 'ciudad_destino': 'Tenerife',
 'salida': '2024-11-03T07:05:00',
 'llegada': '2024-11-03T08:55:00',
 'duracion(min)': 170,
 'escalas': 0,
 'aerolinea': 'Air Europa',
 'precio': 174.66}

In [89]:
# lista_dicct=[]

# for vuelo in range(0,len(resp["itineraries"])):
#         resp= respuesta["data"]
#         precio= resp["itineraries"][0]["price"]["raw"]
#         for i in range(0,2):
#                 id_origen= resp["itineraries"][vuelo]["legs"][i]["origin"]["id"]
#                 origen= resp["itineraries"][vuelo]["legs"][i]["origin"]["city"]

#                 id_destino= resp["itineraries"][vuelo]["legs"][i]["destination"]["id"]
#                 destino= resp["itineraries"][vuelo]["legs"][i]["destination"]["city"]

#                 duracion_minuts= resp["itineraries"][vuelo]["legs"][i]["durationInMinutes"]
#                 escalas= resp["itineraries"][vuelo]["legs"][i]["stopCount"]

#                 salida= resp["itineraries"][vuelo]["legs"][i]["departure"]
#                 llegada= resp["itineraries"][vuelo]["legs"][i]["arrival"]

#                 aerolinea= resp["itineraries"][vuelo]["legs"][i]["carriers"]["marketing"][0]["name"]

#                 dicct= {"id_origen":id_origen,
#                 "ciudad_origen":origen,
#                 "id_destino": id_destino,
#                 "ciudad_destino":destino,
#                 "salida":salida,
#                 "llegada":llegada,
#                 "duracion(min)":duracion_minuts,
#                 "escalas":escalas,
#                 "aerolinea":aerolinea,
#                 "precio":precio
#                 }
#                 lista_dicct.append(dicct)


In [134]:
# df_vuelos_ida_vuelta_TENE= pd.DataFrame(lista_dicct)
# df_vuelos_ida_vuelta_TENE.head(2)

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,TFN,Tenerife,2024-11-03T07:05:00,2024-11-03T08:55:00,170,0,Air Europa,174.66
1,TFN,Tenerife,MAD,Madrid,2024-11-06T17:55:00,2024-11-06T21:40:00,165,0,Air Europa,174.66


In [91]:
# df_vuelos_ida_vuelta_TENE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_origen       20 non-null     object 
 1   ciudad_origen   20 non-null     object 
 2   id_destino      20 non-null     object 
 3   ciudad_destino  20 non-null     object 
 4   salida          20 non-null     object 
 5   llegada         20 non-null     object 
 6   duracion(min)   20 non-null     int64  
 7   escalas         20 non-null     int64  
 8   aerolinea       20 non-null     object 
 9   precio          20 non-null     float64
dtypes: float64(1), int64(2), object(7)
memory usage: 1.7+ KB


Formateo:


In [92]:
# df_vuelos_ida_vuelta_TENE["salida"]= pd.to_datetime(df_vuelos_ida_vuelta_TENE["salida"])
# df_vuelos_ida_vuelta_TENE["llegada"]= pd.to_datetime(df_vuelos_ida_vuelta_TENE["llegada"])
# df_vuelos_ida_vuelta_TENE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_origen       20 non-null     object        
 1   ciudad_origen   20 non-null     object        
 2   id_destino      20 non-null     object        
 3   ciudad_destino  20 non-null     object        
 4   salida          20 non-null     datetime64[ns]
 5   llegada         20 non-null     datetime64[ns]
 6   duracion(min)   20 non-null     int64         
 7   escalas         20 non-null     int64         
 8   aerolinea       20 non-null     object        
 9   precio          20 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(5)
memory usage: 1.7+ KB


**DF vuelos de TENE Done**

In [133]:
# df_vuelos_ida_vuelta_TENE.head(2)

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,TFN,Tenerife,2024-11-03 07:05:00,2024-11-03 08:55:00,170,0,Air Europa,174.66
1,TFN,Tenerife,MAD,Madrid,2024-11-06 17:55:00,2024-11-06 21:40:00,165,0,Air Europa,174.66


web para mirar los códigos:   


https://rapidapi.com/apiheya/api/sky-scrapper/playground/apiendpoint_efcce954-2563-41cb-97fc-b2abe7851b3d

In [ ]:
lista_codigos_aeropuertos= ["MAD", "PMI", "TENE(TFN)","GMZ", "MAH"]

**API de los vuelos(PMI)**  Meter los códigos de aeropuerto que necesitemos

In [4]:
url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {"originSkyId":"MAD","destinationSkyId":"PMI","originEntityId":"95565077","destinationEntityId":"95565111","date":"2024-11-03","returnDate":"2024-11-06","cabinClass":"economy","adults":"1","sortBy":"best","limit":"50","currency":"EUR"}

headers = {
	"x-rapidapi-key": "978309fe9dmshe0626a77c211be6p1786a7jsne7e8845b8cfa",
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

responsa=response.json()


In [73]:


# with open('mallorca.json', 'w') as archivo:
#     json.dump(responsa, archivo, indent=4)

In [98]:

# res2 = res["itineraries"][0]["legs"][0]["origin"]
# # res3 = res["itineraries"][0]["legs"][0]["destination"]



In [74]:
# res= responsa["data"]
# precio= res["itineraries"][0]["price"]["raw"]

# id_origen= res["itineraries"][0]["legs"][0]["origin"]["id"]
# origen= res["itineraries"][0]["legs"][0]["origin"]["city"]

# id_destino= res["itineraries"][0]["legs"][0]["destination"]["id"]
# destino= res["itineraries"][0]["legs"][0]["destination"]["city"]

# duracion_minuts= res["itineraries"][0]["legs"][0]["durationInMinutes"]
# escalas= res["itineraries"][0]["legs"][0]["stopCount"]

# salida= res["itineraries"][0]["legs"][0]["departure"]
# llegada= res["itineraries"][0]["legs"][0]["arrival"]

# aerolinea= res["itineraries"][0]["legs"][0]["carriers"]["marketing"][0]["name"]

In [75]:


# dicc= {"id_origen":id_origen,
#        "ciudad_origen":origen,
#        "id_destino": id_destino,
#        "ciudad_destino":destino,
#        "salida":salida,
#        "llegada":llegada,
#        "duracion(min)":duracion_minuts,
#        "escalas":escalas,
#        "aerolinea":aerolinea,
#        "precio":precio
#         }
# dicc

{'id_origen': 'MAD',
 'ciudad_origen': 'Madrid',
 'id_destino': 'PMI',
 'ciudad_destino': 'Palma',
 'salida': '2024-11-03T08:35:00',
 'llegada': '2024-11-03T10:05:00',
 'duracion(min)': 90,
 'escalas': 0,
 'aerolinea': 'Ryanair',
 'precio': 50.77}

Sacar vuelos ida y vuelta a PMI

In [116]:
# lista_dicc=[]

# for vuelo in range(0,len(res["itineraries"])):
#         res= responsa["data"]
#         precio= res["itineraries"][vuelo]["price"]["raw"]
#         for i in range(0,2):
#                 id_origen= res["itineraries"][vuelo]["legs"][i]["origin"]["id"]
#                 origen= res["itineraries"][vuelo]["legs"][i]["origin"]["city"]

#                 id_destino= res["itineraries"][vuelo]["legs"][i]["destination"]["id"]
#                 destino= res["itineraries"][vuelo]["legs"][i]["destination"]["city"]

#                 duracion_minuts= res["itineraries"][vuelo]["legs"][i]["durationInMinutes"]
#                 escalas= res["itineraries"][vuelo]["legs"][i]["stopCount"]

#                 salida= res["itineraries"][vuelo]["legs"][i]["departure"]
#                 llegada= res["itineraries"][vuelo]["legs"][i]["arrival"]

#                 aerolinea= res["itineraries"][vuelo]["legs"][i]["carriers"]["marketing"][0]["name"]

#                 dicc= {"id_origen":id_origen,
#                 "ciudad_origen":origen,
#                 "id_destino": id_destino,
#                 "ciudad_destino":destino,
#                 "salida":salida,
#                 "llegada":llegada,
#                 "duracion(min)":duracion_minuts,
#                 "escalas":escalas,
#                 "aerolinea":aerolinea,
#                 "precio":precio
#                 }
#                 lista_dicc.append(dicc)


In [132]:
# df_vuelos_ida_vuelta_PMI= pd.DataFrame(lista_dicc)
# df_vuelos_ida_vuelta_PMI.head(2)

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,PMI,Palma,2024-11-03T08:35:00,2024-11-03T10:05:00,90,0,Ryanair,50.77
1,PMI,Palma,MAD,Madrid,2024-11-06T22:20:00,2024-11-06T23:50:00,90,0,Ryanair,50.77


In [118]:
# df_vuelos_ida_vuelta_PMI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_origen       20 non-null     object 
 1   ciudad_origen   20 non-null     object 
 2   id_destino      20 non-null     object 
 3   ciudad_destino  20 non-null     object 
 4   salida          20 non-null     object 
 5   llegada         20 non-null     object 
 6   duracion(min)   20 non-null     int64  
 7   escalas         20 non-null     int64  
 8   aerolinea       20 non-null     object 
 9   precio          20 non-null     float64
dtypes: float64(1), int64(2), object(7)
memory usage: 1.7+ KB


Formateo:


In [119]:
# df_vuelos_ida_vuelta_PMI["salida"]= pd.to_datetime(df_vuelos_ida_vuelta_PMI["salida"])
# df_vuelos_ida_vuelta_PMI["llegada"]= pd.to_datetime(df_vuelos_ida_vuelta_PMI["llegada"])
# df_vuelos_ida_vuelta_PMI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_origen       20 non-null     object        
 1   ciudad_origen   20 non-null     object        
 2   id_destino      20 non-null     object        
 3   ciudad_destino  20 non-null     object        
 4   salida          20 non-null     datetime64[ns]
 5   llegada         20 non-null     datetime64[ns]
 6   duracion(min)   20 non-null     int64         
 7   escalas         20 non-null     int64         
 8   aerolinea       20 non-null     object        
 9   precio          20 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(5)
memory usage: 1.7+ KB


**DF vuelos de PMI Done**

In [120]:
# df_vuelos_ida_vuelta_PMI.head()

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,PMI,Palma,2024-11-03 08:35:00,2024-11-03 10:05:00,90,0,Ryanair,50.77
1,PMI,Palma,MAD,Madrid,2024-11-06 22:20:00,2024-11-06 23:50:00,90,0,Ryanair,50.77
2,MAD,Madrid,PMI,Palma,2024-11-03 08:35:00,2024-11-03 10:05:00,90,0,Ryanair,65.49
3,PMI,Palma,MAD,Madrid,2024-11-06 10:55:00,2024-11-06 12:25:00,90,0,Ryanair,65.49
4,MAD,Madrid,PMI,Palma,2024-11-03 07:05:00,2024-11-03 08:30:00,85,0,Iberia Express,46.60


**Funciones vuelos(ya pasadas a .py):**

In [5]:
def info_vuelos(request_jeison):
    lista_dicc=[]
    res= request_jeison["data"]

    for vuelo in range(0,len(res["itineraries"])): 
        precio= res["itineraries"][vuelo]["price"]["raw"]

        for i in range(0,2):
                id_origen= res["itineraries"][vuelo]["legs"][i]["origin"]["id"]
                origen= res["itineraries"][vuelo]["legs"][i]["origin"]["city"]

                id_destino= res["itineraries"][vuelo]["legs"][i]["destination"]["id"]
                destino= res["itineraries"][vuelo]["legs"][i]["destination"]["city"]

                duracion_minuts= res["itineraries"][vuelo]["legs"][i]["durationInMinutes"]
                escalas= res["itineraries"][vuelo]["legs"][i]["stopCount"]

                salida= res["itineraries"][vuelo]["legs"][i]["departure"]
                llegada= res["itineraries"][vuelo]["legs"][i]["arrival"]

                aerolinea= res["itineraries"][vuelo]["legs"][i]["carriers"]["marketing"][0]["name"]

                dicc= {"id_origen":id_origen,
                "ciudad_origen":origen,
                "id_destino": id_destino,
                "ciudad_destino":destino,
                "salida":salida,
                "llegada":llegada,
                "duracion(min)":duracion_minuts,
                "escalas":escalas,
                "aerolinea":aerolinea,
                "precio":precio
                }
                lista_dicc.append(dicc)
    return lista_dicc

        

In [130]:
# info_vuelos(respuesta)     #Lo menciono para que NO MOLESTE

In [7]:
def crearDF_formateado(funcion_info_vuelos):

    df_vuelos_ida_vuelta= pd.DataFrame(funcion_info_vuelos)
    df_vuelos_ida_vuelta["salida"]= pd.to_datetime(df_vuelos_ida_vuelta["salida"])
    df_vuelos_ida_vuelta["llegada"]= pd.to_datetime(df_vuelos_ida_vuelta["llegada"])
    return df_vuelos_ida_vuelta

In [131]:
#crearDF_formateado(info_vuelos(respuesta))    #Lo menciono para que NO MOLESTE

**Concat de las pags de pmi y tene**  



In [48]:
hotel_tene_1=pd.read_csv("datos/hotel_tene_1.csv")
hotel_tene_2=pd.read_csv("datos/hotel_tene_2.csv")
hotel_tene_3=pd.read_csv("datos/hotel_tene_3.csv")
hotel_tene_4=pd.read_csv("datos/hotel_tene_4.csv")


hotel_pmi_1=pd.read_csv("datos/hotel_pmi_1.csv")
hotel_pmi_2=pd.read_csv("datos/hotel_pmi_2.csv")
hotel_pmi_3=pd.read_csv("datos/hotel_pmi_3.csv")
hotel_pmi_4=pd.read_csv("datos/hotel_pmi_4.csv")
hotel_pmi_5=pd.read_csv("datos/hotel_pmi_5.csv")
hotel_pmi_6=pd.read_csv("datos/hotel_pmi_6.csv")
hotel_pmi_7=pd.read_csv("datos/hotel_pmi_7.csv")
hotel_pmi_8=pd.read_csv("datos/hotel_pmi_8.csv")


In [50]:
df_hoteles_pmi=pd.DataFrame()
df_hoteles_pmi= pd.concat([df_hoteles_pmi,hotel_pmi_1,hotel_pmi_2,hotel_pmi_3,hotel_pmi_4,hotel_pmi_5,hotel_pmi_6,hotel_pmi_7,hotel_pmi_8], ignore_index= True)

In [56]:
df_hoteles_pmi.describe().T


,count,mean,std,min,25%,50%,75%,max
reviews_hotel,216.0,322.356481,615.907421,0.0,2.00,39.0,336.75,3702.0
rating_hotel,216.0,3.537037,1.624956,0.0,3.00,4.0,4.50,5.0
precio_medio_por noche,155.0,230.874194,116.262796,30.0,150.25,219.5,300.25,631.5


In [57]:
df_hoteles_pmi.describe(include="O").T

,count,unique,top,freq
nombre_hotel,216,204,Metropolitan Juka Playa,3
ciudad,216,1,Palma de Mallorca,216
servicios,216,75,[],67
link,216,206,https://www.tripadvisor.com/Hotel_Review-g1874...,2


In [58]:
df_hoteles_tene=pd.DataFrame()
df_hoteles_tene= pd.concat([df_hoteles_tene,hotel_tene_1,hotel_tene_2,hotel_tene_3,hotel_tene_4], ignore_index= True)

In [64]:
df_hoteles_pmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            216 non-null    object 
 1   reviews_hotel           216 non-null    int64  
 2   rating_hotel            216 non-null    float64
 3   precio_medio_por noche  155 non-null    float64
 4   ciudad                  216 non-null    object 
 5   servicios               216 non-null    object 
 6   link                    216 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 11.9+ KB


**Hay datos nulos en el precio medio asi que habrá que gestionarlo en algun momento**  

se pierde un 33,94% de tenerife si se eliminan los nulos     
se pierde un 28,24% de pmi si se eliminan los nulos   
respecto al total, se pierde un 30% respecto al total

Me planteo rellenarlo con la media

In [59]:
df_hoteles_tene.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            109 non-null    object 
 1   reviews_hotel           109 non-null    int64  
 2   rating_hotel            109 non-null    float64
 3   precio_medio_por noche  72 non-null     float64
 4   ciudad                  109 non-null    object 
 5   servicios               109 non-null    object 
 6   link                    109 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 6.1+ KB


In [60]:
df_hoteles_tene.describe().T

,count,mean,std,min,25%,50%,75%,max
reviews_hotel,109.0,108.733945,374.916673,0.0,0.000,3.0,32.000,2829.0
rating_hotel,109.0,2.518349,1.995280,0.0,0.000,3.5,4.000,5.0
precio_medio_por noche,72.0,92.027778,36.722397,26.0,63.125,88.5,121.125,178.5


In [61]:
df_hoteles_tene.describe(include="O").T

,count,unique,top,freq
nombre_hotel,109,102,Montes de Anaga Hostel,2
ciudad,109,1,Santa Cruz de Tenerife,109
servicios,109,26,[],61
link,109,102,https://www.tripadvisor.com/Hotel_Review-g1874...,2


In [65]:
df_hoteles_total= pd.DataFrame()
df_hoteles_total=pd.concat([df_hoteles_total,df_hoteles_pmi,df_hoteles_tene], ignore_index= True)

In [66]:
df_hoteles_total.head()

,nombre_hotel,reviews_hotel,rating_hotel,precio_medio_por noche,ciudad,servicios,link
0,Hotel Costa Azul,2081,4.5,177.0,Palma de Mallorca,"['Free Wifi', 'Pool', 'Indoor pool', 'Restaura...",https://www.tripadvisor.com/Hotel_Review-g1874...
1,Hotel Victoria Gran Meliá,2447,4.5,270.0,Palma de Mallorca,"['Free Wifi', 'Pool', 'Heated pool', 'Indoor p...",https://www.tripadvisor.com/Hotel_Review-g1874...
2,Nakar Hotel,2327,5.0,349.5,Palma de Mallorca,"['Free Wifi', 'Breakfast included', 'Pool', 'H...",https://www.tripadvisor.com/Hotel_Review-g1874...
3,INNSiDE Palma Bosque,1082,4.5,160.0,Palma de Mallorca,"['Free Wifi', 'Indoor pool', 'Outdoor pool', '...",https://www.tripadvisor.com/Hotel_Review-g1874...
4,BQ Belvedere Hotel,3702,4.0,154.0,Palma de Mallorca,"['Free parking', 'Pool', 'Outdoor pool', 'Rest...",https://www.tripadvisor.com/Hotel_Review-g1874...


In [68]:
df_hoteles_total.shape

(325, 7)

In [69]:
df_hoteles_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre_hotel            325 non-null    object 
 1   reviews_hotel           325 non-null    int64  
 2   rating_hotel            325 non-null    float64
 3   precio_medio_por noche  227 non-null    float64
 4   ciudad                  325 non-null    object 
 5   servicios               325 non-null    object 
 6   link                    325 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 17.9+ KB


Los describes no me sirven en el total porque lo que quiero es comparar. (Para visualizar haré group by)

In [70]:
df_hoteles_total.describe()

,reviews_hotel,rating_hotel,precio_medio_por noche
count,325.000000,325.000000,227.000000
mean,250.710769,3.195385,186.834802
std,555.681044,1.819682,117.592932
min,0.000000,0.000000,26.000000
25%,1.000000,2.000000,96.500000
50%,14.000000,4.000000,159.000000
75%,198.000000,4.500000,254.000000
max,3702.000000,5.000000,631.500000
